<a href="https://colab.research.google.com/github/NBene10/DrugCrossDomainSentimentAnalysis/blob/main/Cross_Domain_SentimentAnlysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
!pip install vadersentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
import nltk


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI/datasets/drugsComTrain_raw.tsv", delimiter="\t",encoding='utf-8')

In [ ]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [ ]:

print("Dataframe total NaN values: ")
df.isnull().T.any().T.sum()


Dataframe total NaN values: 


899

In [ ]:
#dataset removing NaN value
df.fillna('', inplace=True)
print("Dataframe total NaN values: ")
df.isnull().T.any().T.sum()

Dataframe total NaN values: 


0

In [ ]:
#text pre-processing dataset column 'review'

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):

    #Remove non-alphanumeric characters using regular expression and convert to
    #lowercase
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())

    #Tokenizing and removing stop-words
    stop_words = set(stopwords.words('english')) #set di stop-words
    words = word_tokenize(text) #divide text into tokens
    #maintain only tokens that not being inside stop_words set
    filtered_words = [word for word in words if word not in stop_words]

    #unite singol filtered token into a unique string, separating them by
    #a space
    return ' '.join(filtered_words)

df['cleaned_review'] = df['review'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df.head(10)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,son halfway fourth week intuniv became concern...
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,used take another oral contraceptive 21 pill c...
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,first time using form birth control 039 glad w...
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,suboxone completely turned life around feel he...
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,2nd day 5mg started work rock hard erections h...
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0,"March 7, 2017",5,pulled cummed bit took plan b 26 hours later t...
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10.0,"March 14, 2015",32,abilify changed life hope zoloft clonidine fir...
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0,"August 9, 2016",11,nothing problems keppera constant shaking arms...
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8.0,"December 8, 2016",1,pill many years doctor changed rx chateal effe...


In [ ]:
analyser = SentimentIntensityAnalyzer()

#analyser for cleaned review
df['cleaned_review-dict-VD'] = df['cleaned_review'].apply(analyser.polarity_scores)
#transform VADER dictionary into a number list of the corresponding
#values of the keys
df['cleaned_review-numberlist-VD'] = df['cleaned_review-dict-VD'].apply(lambda x: x.values())

#analyser for standard review
df['standard_review-dict-VD'] = df['review'].apply(analyser.polarity_scores)
df['standard_review-numberlist-VD'] = df['standard_review-dict-VD'].apply(lambda x: x.values())

dfcleanVD = df['cleaned_review-numberlist-VD']
dfstandardVD = df['standard_review-numberlist-VD']

dfcleanVD = pd.DataFrame(dfcleanVD.tolist(), columns=['key_neg', 'key_neu', 'key_pos', 'key_compound'])
dfstandardVD = pd.DataFrame(dfstandardVD.tolist(), columns=['key_neg', 'key_neu', 'key_pos', 'key_compound'])


In [ ]:
df.head(10)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review,cleaned_review-dict-VD,cleaned_review-numberlist-VD,standard_review-dict-VD,standard_review-numberlist-VD
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)","{'neg': 0.121, 'neu': 0.879, 'pos': 0.0, 'comp...","(0.121, 0.879, 0.0, -0.296)"
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,son halfway fourth week intuniv became concern...,"{'neg': 0.035, 'neu': 0.759, 'pos': 0.206, 'co...","(0.035, 0.759, 0.206, 0.9068)","{'neg': 0.018, 'neu': 0.874, 'pos': 0.108, 'co...","(0.018, 0.874, 0.108, 0.9174)"
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,used take another oral contraceptive 21 pill c...,"{'neg': 0.062, 'neu': 0.811, 'pos': 0.126, 'co...","(0.062, 0.811, 0.126, 0.7506)","{'neg': 0.059, 'neu': 0.861, 'pos': 0.08, 'com...","(0.059, 0.861, 0.08, 0.616)"
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,first time using form birth control 039 glad w...,"{'neg': 0.039, 'neu': 0.804, 'pos': 0.157, 'co...","(0.039, 0.804, 0.157, 0.7184)","{'neg': 0.026, 'neu': 0.885, 'pos': 0.089, 'co...","(0.026, 0.885, 0.089, 0.7184)"
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,suboxone completely turned life around feel he...,"{'neg': 0.105, 'neu': 0.612, 'pos': 0.283, 'co...","(0.105, 0.612, 0.283, 0.9344)","{'neg': 0.061, 'neu': 0.771, 'pos': 0.168, 'co...","(0.061, 0.771, 0.168, 0.9403)"
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,2nd day 5mg started work rock hard erections h...,"{'neg': 0.217, 'neu': 0.661, 'pos': 0.121, 'co...","(0.217, 0.661, 0.121, -0.6249)","{'neg': 0.167, 'neu': 0.754, 'pos': 0.079, 'co...","(0.167, 0.754, 0.079, -0.4342)"
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0,"March 7, 2017",5,pulled cummed bit took plan b 26 hours later t...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)"
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10.0,"March 14, 2015",32,abilify changed life hope zoloft clonidine fir...,"{'neg': 0.292, 'neu': 0.591, 'pos': 0.116, 'co...","(0.292, 0.591, 0.116, -0.9468)","{'neg': 0.144, 'neu': 0.78, 'pos': 0.077, 'com...","(0.144, 0.78, 0.077, -0.9089)"
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0,"August 9, 2016",11,nothing problems keppera constant shaking arms...,"{'neg': 0.165, 'neu': 0.691, 'pos': 0.144, 'co...","(0.165, 0.691, 0.144, -0.1386)","{'neg': 0.192, 'neu': 0.7, 'pos': 0.108, 'comp...","(0.192, 0.7, 0.108, -0.5593)"
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8.0,"December 8, 2016",1,pill many years doctor changed rx chateal effe...,"{'neg': 0.202, 'neu': 0.513, 'pos': 0.284, 'co...","(0.202, 0.513, 0.284, 0.7876)","{'neg': 0.167, 'neu': 0.669, 'pos': 0.164, 'co...","(0.167, 0.669, 0.164, -0.5593)"


In [ ]:
dfcleanVD.head()

,key_neg,key_neu,key_pos,key_compound
0,0.000,1.000,0.000,0.0000
1,0.035,0.759,0.206,0.9068
2,0.062,0.811,0.126,0.7506
3,0.039,0.804,0.157,0.7184
4,0.105,0.612,0.283,0.9344


In [ ]:
dfstandardVD.head()

,key_neg,key_neu,key_pos,key_compound
0,0.121,0.879,0.000,-0.2960
1,0.018,0.874,0.108,0.9174
2,0.059,0.861,0.080,0.6160
3,0.026,0.885,0.089,0.7184
4,0.061,0.771,0.168,0.9403


In [ ]:
dfclean_final = pd.merge(df, dfcleanVD, left_index=True, right_index=True)
dfclean_final.drop(columns=['standard_review-dict-VD'], inplace=True)
dfclean_final.drop(columns=['standard_review-numberlist-VD'], inplace=True)

In [ ]:
dfclean_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review,cleaned_review-dict-VD,cleaned_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)",0.0,1.0,0.0,0.0


In [ ]:
dfstandard_final = pd.merge(df, dfstandardVD, left_index=True, right_index=True)
dfstandard_final.drop(columns=['cleaned_review'], inplace=True)
dfstandard_final.drop(columns=['cleaned_review-dict-VD'], inplace=True)
dfstandard_final.drop(columns=['cleaned_review-numberlist-VD'], inplace=True)

In [ ]:
dfstandard_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,standard_review-dict-VD,standard_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,"{'neg': 0.121, 'neu': 0.879, 'pos': 0.0, 'comp...","(0.121, 0.879, 0.0, -0.296)",0.121,0.879,0.0,-0.296


In [ ]:

#function that maps values of "key_compound" column in a proper range
def map_rating_model(key_compound):
    if -1 <= key_compound <= -0.3:
        return 0 #negative sentiment
    elif -0.3 < key_compound < 0.3:
        return 1 #neutral sentiment
    elif key_compound >= 0.3:
        return 2 #positive sentiment
    else:
        return None  #for unexpected values


#apply map_rating_model on "rating" column and so create new "rating_model"
#column
dfclean_final['rating_model'] = dfclean_final['key_compound'].apply(lambda x: map_rating_model(x))
dfstandard_final['rating_model'] = dfstandard_final['key_compound'].apply(lambda x: map_rating_model(x))


In [ ]:
dfclean_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review,cleaned_review-dict-VD,cleaned_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound,rating_model
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)",0.0,1.0,0.0,0.0,1


In [ ]:
dfstandard_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,standard_review-dict-VD,standard_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound,rating_model
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,"{'neg': 0.121, 'neu': 0.879, 'pos': 0.0, 'comp...","(0.121, 0.879, 0.0, -0.296)",0.121,0.879,0.0,-0.296,1


In [ ]:
conditions=['Birth Control', 'Depression','Pain', 'Anxiety', 'Diabetes, Type 2']

#model creation

def model_data_clean(condition):
    dfclean_CrossDomain = dfclean_final[dfclean_final['condition'] == condition].copy()
    print(len(dfclean_CrossDomain))
    X_trainCl, X_testCl, y_trainCl, y_testCl = train_test_split(dfclean_CrossDomain['cleaned_review'], dfclean_CrossDomain['rating_model'], test_size=0.33, random_state=42)
    return X_trainCl, X_testCl, y_trainCl, y_testCl

def model_data_standard(condition):
    dfstandard_CrossDomain = dfstandard_final[dfstandard_final['condition'] == condition].copy()
    print(len(dfstandard_CrossDomain))
    X_trainSt, X_testSt, y_trainSt, y_testSt = train_test_split(dfstandard_CrossDomain['review'], dfstandard_CrossDomain['rating_model'], test_size=0.33, random_state=42)
    return X_trainSt, X_testSt, y_trainSt, y_testSt


In [ ]:
#model variables for cleaned dataset

#define model variables for "Birth Control" domain
X_BC_trainCl, X_BC_testCl, y_BC_trainCl, y_BC_testCl = model_data_clean('Birth Control')

#define model variables for "Depression" domain
X_D_trainCl, X_D_testCl, y_D_trainCl, y_D_testCl = model_data_clean('Depression')

#define model variables for "Pain" domain
X_P_trainCl, X_P_testCl, y_P_trainCl, y_P_testCl = model_data_clean('Pain')

#define model variables for "Anxiety" domain
X_A_trainCl, X_A_testCl, y_A_trainCl, y_A_testCl = model_data_clean('Anxiety')

#define model variables for "Diabetes Type 2" domain
X_Dia_trainCl, X_Dia_testCl, y_Dia_trainCl, y_Dia_testCl = model_data_clean('Diabetes, Type 2')

28788
9069
6145
5904
2554


In [ ]:
#model variables for standard dataset

#define model variables for "Birth Control" domain
X_BC_trainSt, X_BC_testSt, y_BC_trainSt, y_BC_testSt = model_data_standard('Birth Control')

#define model variables for "Depression" domain
X_D_trainSt, X_D_testSt, y_D_trainSt, y_D_testSt = model_data_standard('Depression')

#define model variables for "Pain" domain
X_P_trainSt, X_P_testSt, y_P_trainSt, y_P_testSt = model_data_standard('Pain')

#define model variables for "Anxiety" domain
X_A_trainSt, X_A_testSt, y_A_trainSt, y_A_testSt = model_data_standard('Anxiety')

#define model variables for "Diabetes Tyoe 2" domain
X_Dia_trainSt, X_Dia_testSt, y_Dia_trainSt, y_Dia_testSt = model_data_standard('Diabetes, Type 2')

28788
9069
6145
5904
2554


In [ ]:
#define necessary strings for the output of Perceptron()
trainCl_str_Inputs = [['X_BC_trainCl', 'y_BC_trainCl'],['X_D_trainCl', 'y_D_trainCl'], ['X_P_trainCl','y_P_trainCl'], ['X_A_trainCl','y_A_trainCl'],['X_Dia_trainCl','y_Dia_trainCl']]
testCl_str_Inputs = [['X_BC_testCl','y_BC_testCl'], ['X_D_testCl','y_D_testCl'],['X_P_testCl','y_P_tesCl'],['X_A_testCl','y_A_testCl'],['X_Dia_testCl','y_Dia_testCl']]
trainSt_str_Inputs = [['X_BC_trainSt', 'y_BC_trainSt'],['X_D_trainSt', 'y_D_trainSt'], ['X_P_trainSt','y_P_trainSt'], ['X_A_trainSt','y_A_trainSt'],['X_Dia_trainSt','y_Dia_trainSt']]
testSt_str_Inputs = [['X_BC_testSt','y_BC_testSt'], ['X_D_testSt','y_D_testSt'],['X_P_testSt','y_P_tesSt'],['X_A_testSt','y_A_testSt'],['X_Dia_testSt','y_Dia_testSt']]


string_combinationCl= [t+j for j in testCl_str_Inputs for t in trainCl_str_Inputs]
#list comprehension in Python that generate all possible combinations between
#"testCl_str_Inputs" elements and "trainCl_str_Inputs" elements, concatenating
#them inside "string_combinationCl"

string_combinationSt= [t+j for j in testSt_str_Inputs for t in trainSt_str_Inputs]

#we have to consider all possible combinations of the model variables of
#the every domain for executing cross-domain analysis
trainCl_var_Inputs= [(X_BC_trainCl, y_BC_trainCl),(X_D_trainCl, y_D_trainCl), (X_P_trainCl,y_P_trainCl), (X_A_trainCl,y_A_trainCl),(X_Dia_trainCl,y_Dia_trainCl)]
testCl_var_Inputs = [(X_BC_testCl,y_BC_testCl), (X_D_testCl,y_D_testCl),(X_P_testCl,y_P_testCl),(X_A_testCl,y_A_testCl),(X_Dia_testCl,y_Dia_testCl)]
varCl_combination= [t+j for j in testCl_var_Inputs for t in trainCl_var_Inputs]

trainSt_var_Inputs= [(X_BC_trainSt, y_BC_trainSt),(X_D_trainSt, y_D_trainSt), (X_P_trainSt,y_P_trainSt), (X_A_trainSt,y_A_trainSt),(X_Dia_trainSt,y_Dia_trainSt)]
testSt_var_Inputs = [(X_BC_testSt,y_BC_testSt), (X_D_testSt,y_D_testSt),(X_P_testSt,y_P_testSt),(X_A_testSt,y_A_testSt),(X_Dia_testSt,y_Dia_testSt)]
varSt_combination= [t+j for j in testSt_var_Inputs for t in trainSt_var_Inputs]

In [ ]:
#list of possible classifiers
Classifiers = [
    Perceptron(),
]

#instance of TfidVectorizer class where we have
#analysis unity: word
#range of words considered: uni-gram and bi-gram
v = TfidfVectorizer(analyzer="word", ngram_range=(1, 2))

dicti_Cl = {"train": [], "test": [], "accuracy": []}
dicti_St = {"train": [], "test": [], "accuracy": []}

print("[PERCEPTRON ON CLEAN DF]")
for i, (X_trainCl, y_trainCl, X_testCl, y_testCl) in zip(string_combinationCl, varCl_combination):
    print('-' * 100)
    print(i)
    print('-' * 100)

    #features extraction and train model v
    train_featuresCl = v.fit_transform(X_trainCl) #train model v with "X_trainCl" data
                                                  #"train_featuresCl" now are
                                                  #features vector TF-IDF and
                                                  #became train data for our
                                                  #model
    test_featuresCl = v.transform(X_testCl) #transform test data X_testCl using
                                            #trained model v

    if train_featuresCl.shape[1] != test_featuresCl.shape[1]:
        print(f"Error: Dimension mismatch between training and test features for {i}")
        continue

    for classifier in Classifiers:
        try:
            fit = classifier.fit(train_featuresCl, y_trainCl) #train model with Perceptron
            pred = fit.predict(test_featuresCl) #Perceptron prediction
            accuracy = accuracy_score(pred, y_testCl)

            # outcome record on the dictionary
            dicti_Cl["train"].append(i[0][2:4])
            dicti_Cl["test"].append(i[3][2:4])
            dicti_Cl["accuracy"].append(float(accuracy))
            print("Accuracy of " + classifier.__class__.__name__ + " is " + str(accuracy))
        except Exception as e:
            print(f"Error: {str(e)}")

print("\n[PERCEPTRON ON STANDARD DF]")
for i, (X_trainSt, y_trainSt, X_testSt, y_testSt) in zip(string_combinationSt, varSt_combination):
    print('-' * 100)
    print(i)
    print('-' * 100)


    train_featuresSt = v.fit_transform(X_trainSt)
    test_featuresSt = v.transform(X_testSt)

    for classifier in Classifiers:
        try:
            fit = classifier.fit(train_featuresSt, y_trainSt)
            pred = fit.predict(test_featuresSt)
            accuracy = accuracy_score(pred, y_testSt)

            # Registrazione dei Risultati nel Dizionario
            dicti_St["train"].append(i[0][2:4])
            dicti_St["test"].append(i[3][2:4])
            dicti_St["accuracy"].append(float(accuracy))
            print("Accuracy of " + classifier.__class__.__name__ + " is " + str(accuracy))
        except Exception as e:
            print(f"Error: {str(e)}")



[PERCEPTRON ON CLEAN DF]
----------------------------------------------------------------------------------------------------
['X_BC_trainCl', 'y_BC_trainCl', 'X_BC_testCl', 'y_BC_testCl']
----------------------------------------------------------------------------------------------------
Accuracy of Perceptron is 0.875381538785391
----------------------------------------------------------------------------------------------------
['X_D_trainCl', 'y_D_trainCl', 'X_BC_testCl', 'y_BC_testCl']
----------------------------------------------------------------------------------------------------
Accuracy of Perceptron is 0.6879275865698348
----------------------------------------------------------------------------------------------------
['X_P_trainCl', 'y_P_trainCl', 'X_BC_testCl', 'y_BC_testCl']
----------------------------------------------------------------------------------------------------
Accuracy of Perceptron is 0.6522471318808546
--------------------------------------------------

In [ ]:
print(dicti_Cl)

{'train': ['BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di'], 'test': ['BC', 'BC', 'BC', 'BC', 'BC', 'D_', 'D_', 'D_', 'D_', 'D_', 'P_', 'P_', 'P_', 'P_', 'P_', 'A_', 'A_', 'A_', 'A_', 'A_', 'Di', 'Di', 'Di', 'Di', 'Di'], 'accuracy': [0.875381538785391, 0.6879275865698348, 0.6522471318808546, 0.6769813703820651, 0.6383538574886854, 0.7530905446040762, 0.8503174072836619, 0.6555295689943201, 0.7109923154026061, 0.6501837621115937, 0.7006903353057199, 0.6893491124260355, 0.8131163708086785, 0.6785009861932939, 0.6464497041420119, 0.7316572601334017, 0.7075423293996922, 0.6367367880964597, 0.8009235505387378, 0.6126218573627501, 0.6963226571767497, 0.6666666666666666, 0.6251482799525504, 0.6595492289442467, 0.7924080664294187]}


In [ ]:
print(dicti_St)

{'train': ['BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di'], 'test': ['BC', 'BC', 'BC', 'BC', 'BC', 'D_', 'D_', 'D_', 'D_', 'D_', 'P_', 'P_', 'P_', 'P_', 'P_', 'A_', 'A_', 'A_', 'A_', 'A_', 'Di', 'Di', 'Di', 'Di', 'Di'], 'accuracy': [0.8569624250078939, 0.6525628881170403, 0.6075149984212188, 0.6225660456794022, 0.6129881065151037, 0.6782492482459072, 0.821917808219178, 0.6348145673237554, 0.6495155362512529, 0.5867023053792182, 0.6597633136094675, 0.631163708086785, 0.7840236686390533, 0.5922090729783037, 0.6104536489151874, 0.6516162134427912, 0.6598255515649051, 0.5987685992816829, 0.7650076962544895, 0.5746536685479733, 0.6559905100830368, 0.6014234875444839, 0.5895610913404508, 0.604982206405694, 0.7449584816132859]}


In [ ]:
cross_domain = pd.DataFrame.from_dict(dicti_Cl) #the "cross_domain" DataFrame is created using the data contained in the "dicti_Cl" dictionary
cross_domain['test'].unique()

abbreviation = ['BC', 'D_', 'P_', 'A_', 'Di']

cross_domain.replace(to_replace=abbreviation, value=conditions, inplace=True) #replaces the abbreviations present in the "cross_domain" DataFrame with the full names of
                                                                              #the domains, using the "abbreviation" and "conditions" lists.

In [ ]:
cross_domain_perceptron = cross_domain.pivot_table(values='accuracy', index= 'train', columns='test')

cross_domain_perceptron

test,Anxiety,Birth Control,Depression,"Diabetes, Type 2",Pain
train,,,,,
Anxiety,0.800924,0.676981,0.710992,0.659549,0.678501
Birth Control,0.731657,0.875382,0.753091,0.696323,0.700690
Depression,0.707542,0.687928,0.850317,0.666667,0.689349
"Diabetes, Type 2",0.612622,0.638354,0.650184,0.792408,0.646450
Pain,0.636737,0.652247,0.655530,0.625148,0.813116


In [ ]:
cross_domain_perceptron_s = cross_domain_perceptron.reindex(index=['Depression', 'Anxiety','Pain', 'Birth Control','Diabetes, Type 2' ],columns= ['Depression', 'Anxiety','Pain', 'Birth Control','Diabetes, Type 2' ])
cross_domain_perceptron_s['|Train(Avg. accuracy)|'] = cross_domain_perceptron_s[cross_domain_perceptron_s.columns[::-1]].mean(axis=1) #evaluate average for train accuracies
cross_domain_perceptron_s.loc['|Test(Avg. accuracy)|'] = cross_domain_perceptron_s[cross_domain_perceptron_s.columns[::-1]].mean(axis=0) #evaluate average for test accuracies
cross_domain_perceptron_s

test,Depression,Anxiety,Pain,Birth Control,"Diabetes, Type 2",|Train(Avg. accuracy)|
train,,,,,,
Depression,0.850317,0.707542,0.689349,0.687928,0.666667,0.720361
Anxiety,0.710992,0.800924,0.678501,0.676981,0.659549,0.705389
Pain,0.655530,0.636737,0.813116,0.652247,0.625148,0.676556
Birth Control,0.753091,0.731657,0.700690,0.875382,0.696323,0.751428
"Diabetes, Type 2",0.650184,0.612622,0.646450,0.638354,0.792408,0.668003
|Test(Avg. accuracy)|,0.724023,0.697896,0.705621,0.706178,0.688019,0.704348


In [ ]:
cross_domain= pd.DataFrame.from_dict(dicti_St)
cross_domain['test'].unique()

abbreviation= ['BC', 'D_', 'P_', 'A_', 'Di']

cross_domain.replace(to_replace=abbreviation, value=conditions,inplace=True)

In [ ]:
cross_domain_perceptron = cross_domain.pivot_table(values='accuracy', index= 'train', columns='test')

cross_domain_perceptron

test,Anxiety,Birth Control,Depression,"Diabetes, Type 2",Pain
train,,,,,
Anxiety,0.765008,0.622566,0.649516,0.604982,0.592209
Birth Control,0.651616,0.856962,0.678249,0.655991,0.659763
Depression,0.659826,0.652563,0.821918,0.601423,0.631164
"Diabetes, Type 2",0.574654,0.612988,0.586702,0.744958,0.610454
Pain,0.598769,0.607515,0.634815,0.589561,0.784024
